In [0]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
import mlflow
import mlflow.sklearn


url = 'https://github.com/anfisbena/MIAD-DSA/raw/refs/heads/main/data/RTOP.csv'
df = pd.read_csv(url, encoding='ISO-8859-1')
df = df.rename(columns={'Ship ID - Ship Classification': 'Vessel Type', 'Ship - Name': 'Ship Name'})

def clean_df(df):
    df_clean = df[['Ship Name', 'Vessel Type', 'Country', 'Location', 'Arrival', 'Departure', 'Status']]
    df_clean = df_clean.dropna(subset=['Arrival', 'Departure', 'Ship Name', 'Location', 'Vessel Type', 'Status'])
    df_clean = df_clean[df_clean['Status'] != 'Cancelled']
    # Convertir las columnas 'Arrival' y 'Departure' a datetime
    df_clean['Arrival'] = pd.to_datetime(df_clean['Arrival'], errors='coerce')
    df_clean['Departure'] = pd.to_datetime(df_clean['Departure'], errors='coerce')
    df_clean['Duration'] = (df_clean['Departure'] - df_clean['Arrival']).dt.days
    # Filtrar las filas donde 'Duration' sea menor o igual a 60 días
    df_clean = df_clean[(df_clean['Duration'] <= 60) & (df_clean['Duration'] >= 0)]
    return df_clean

def improved_predict_next_location_classification(df, n_e, mx_dp, lr, ship_name='St. Louis'):
    # Filtrar los datos del barco especificado
    df_ship = df[df['Ship Name'] == ship_name].copy()

    # Convertir columnas de fechas y agregar variables temporales
    df_ship['Arrival_ordinal'] = df_ship['Arrival'].map(pd.Timestamp.toordinal)
    df_ship['Departure_ordinal'] = df_ship['Departure'].map(pd.Timestamp.toordinal)
    df_ship['Arrival_Month'] = df_ship['Arrival'].dt.month
    df_ship['Arrival_Year'] = df_ship['Arrival'].dt.year
    df_ship['Arrival_DayOfWeek'] = df_ship['Arrival'].dt.dayofweek

    # Verificar que la variable objetivo 'Location' esté en el DataFrame
    if 'Location' not in df_ship.columns:
        print("La columna 'Location' no está disponible en el DataFrame.")
        return None

    # Variables categóricas para One-Hot Encoding
    categorical_features = ['Country']
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    # Column Transformer para preprocesar características
    preprocessor = ColumnTransformer(
        transformers=[('cat', categorical_transformer, categorical_features)],
        remainder='passthrough'
    )

    # Convertir la variable objetivo 'Location' a categorías numéricas
    label_encoder = LabelEncoder()
    df_ship['Location'] = label_encoder.fit_transform(df_ship['Location'])

    # Definir características (X) y variable objetivo (y) - 'Location'
    X = df_ship[['Country', 'Arrival_ordinal', 'Departure_ordinal', 'Arrival_Month', 'Arrival_Year', 'Arrival_DayOfWeek']]
    y = df_ship['Location']

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Crear pipeline con preprocesamiento y clasificador Gradient Boosting
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingClassifier(n_estimators=n_e, max_depth=mx_dp, learning_rate=lr, random_state=42))
    ])

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Evaluación del modelo
    y_pred = model.predict(X_test)
    location_accuracy = accuracy_score(y_test, y_pred)
    location_precision = precision_score(y_test, y_pred, average='weighted')
    location_recall = recall_score(y_test, y_pred, average='weighted')
    location_f1 = f1_score(y_test, y_pred, average='weighted')

    # Decodificar la predicción para la próxima ubicación
    predicted_location = label_encoder.inverse_transform([y_pred[-1]])[0]

    print(f"Precisión de Ubicación: {location_accuracy * 100:.2f}%")
    print(f"Precisión (Weighted): {location_precision:.2f}")
    print(f"Recall (Weighted): {location_recall:.2f}")
    print(f"F1 Score (Weighted): {location_f1:.2f}")

    return predicted_location, model, location_accuracy, location_precision, location_recall, location_f1

experiment = mlflow.set_experiment("/Users/republicacoc@gmail.com/RandClass")

n_estimators_list = [50, 100, 500, 1000]
max_depth_list = [2, 5, 10, 15]
learning_rate_list = [0.01, 0.1, 1]
for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        for learning_rate in learning_rate_list:
            predicted_location, model, accuracy, precision, recall, f1 = improved_predict_next_location_classification(clean_df(df), n_estimators, max_depth, learning_rate, ship_name='Trenton')
            with mlflow.start_run(experiment_id=experiment.experiment_id):
                # Registre los parámetros
                mlflow.log_param("num_trees", n_estimators)
                mlflow.log_param("maxdepth", max_depth)
                mlflow.log_param("learning_rate", learning_rate)
                mlflow.log_param("predicted location", predicted_location)
                
                # Registre el modelo
                mlflow.sklearn.log_model(model, "gradient-boosting-model")
                
                # Cree y registre las métricas de interés
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)


/databricks/python/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Uploading artifacts:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

/databricks/python/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precisión de Ubicación: 40.00%
Precisión (Weighted): 0.24
Recall (Weighted): 0.40
F1 Score (Weighted): 0.28


...

Precisión de Ubicación: 35.00%
Precisión (Weighted): 0.25
Recall (Weighted): 0.35
F1 Score (Weighted): 0.28


2024/11/11 03:21:01 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

2024/11/11 03:21:03 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/11/11 03:21:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run wise-squid-791 at: https://community.cloud.databricks.com/ml/experiments/3923292881806332/runs/6f532d7c2f2d4308b0c03fe337bd030f.
2024/11/11 03:21:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/3923292881806332.


---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
File <command-4434647190195530>, line 108
    105 mlflow.log_param("predicted location", predicted_location)
    107 # Registre el modelo
--> 108 mlflow.sklearn.log_model(model, "gradient-boosting-model")
    110 # Cree y registre las métricas de interés
    111 mlflow.log_metric("accuracy", accuracy)

File /databricks/python/lib/python3.11/site-packages/mlflow/sklearn/__init__.py:412, in log_model(sk_model, artifact_path, conda_env, code_paths, serialization_format, registered_model_name, signature, input_example, await_registration_for, pip_requirements, extra_pip_requirements, pyfunc_predict_fn, metadata)
    333 @format_docstring(LOG_MODEL_PARAM_DOCS.format(package_name="scikit-learn"))
    334 def log_model(
    335     sk_model,
   (...)
    347     metadata=None,
    348 ):
    349     """
    350     Log a scikit-learn model as 